In [1]:
import numpy as np
from time import time
import pandas as pd
# #from nltk.tokenize import TweetTokenizer
# from keras.layers import Dense, Embedding, Input, LSTM, GRU, Conv1D, Bidirectional, GlobalMaxPool1D, Dropout
# from keras.layers import SpatialDropout1D, GlobalAveragePooling1D, GlobalMaxPooling1D, concatenate, BatchNormalization
# from keras.preprocessing import text, sequence
from keras.layers import Dense,Input,LSTM,Bidirectional,Activation,Conv1D,GRU
from keras.callbacks import Callback
from keras.layers import Dropout,Embedding,GlobalMaxPooling1D, MaxPooling1D, Add, Flatten
from keras.preprocessing import text, sequence
from keras.layers import GlobalAveragePooling1D, GlobalMaxPooling1D, concatenate, SpatialDropout1D
from keras import initializers, regularizers, constraints, optimizers, layers, callbacks
from keras.callbacks import EarlyStopping,ModelCheckpoint
from keras.models import Model
from keras.optimizers import Adam

from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

HOME = '../../'
DATA = HOME + 'data/'
MODEL = HOME + 'model/'

PATH = '~/data/toxic/data/'
# PATH = '../../data/'

label_cols = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

/home/kai/anaconda3/envs/tf_gpu/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
from fastText import load_model
# grid search params
from sklearn.utils import shuffle
import pandas as pd
import json

class grid_search_generator(object):
    def __init__(self, config_file_url='grid_search_config.json',
                 params=None, bindings=None, score_name_list=None, csv_url=None):
        new_generator = params != None and bindings != None and score_name_list != None
        if new_generator:
            self.params = params
            self.binding = bindings
            self.score_name_list = score_name_list
            
            self.single_keys = [key for key in self.params.keys() if key not in self.binding.keys()]
            self.binding_keys = list(self.binding.keys())
            self._terminate = False
            
            self._save_config(config_file_url)
        else:
            with open(config_file_url, 'r') as f:
                data = json.load(f)
                self.params = data['params']
                self.binding = data['binding']
                self.score_name_list = data['score_name_list']
                self.single_keys = data['single_keys']
                self.binding_keys = data['binding_keys']
                
        self.keys = list(self.single_keys)
        self.keys.extend(self.binding_keys)
        self.param_keys = list(self.single_keys)
        for i in range(len(self.single_keys), len(self.keys)):
            self.param_keys.extend(self.binding[self.keys[i]])
        
        self._terminate = False
        self._idx = [0 for i in range(len(self.keys))]
        self._ub = [len(self.params[key]) for key in self.keys]
        
        if csv_url != None: self.get_csv(csv_url)
    
    def _reset_config(self):
        self._terminate = False
        self._idx = [0 for i in range(len(self.keys))]
        self._ub = [len(self.params[key]) for key in self.keys]
    
    def _save_config(self, config_file_url):
        with open(config_file_url, 'w') as f:
            data = {
                'params': self.params,
                'binding': self.binding,
                'score_name_list': self.score_name_list,
                'single_keys': self.single_keys,
                'binding_keys': self.binding_keys
            }
            json.dump(data, f)
    
    def _next_idx(self):
        i = 0
        self._idx[i] = (self._idx[i] + 1) % self._ub[i]
        i += 1
        end_loop = (self._idx[i-1]!=0 or i==len(self._idx))
        terminate = (self._idx[i-1]==0 and i==len(self._idx))
        while(not end_loop):
            self._idx[i] = (self._idx[i] + 1) % self._ub[i]
            i += 1
            end_loop = (self._idx[i-1]!=0 or i==len(self._idx))
            terminate = (self._idx[i-1]==0 and i==len(self._idx))
        self._terminate = terminate

    def _next_param_list(self):
        if not self._terminate:
            input = [self.params[self.keys[i]][self._idx[i]] for i in range(len(self.single_keys))]
            for i in range(len(self.single_keys), len(self.keys)):
                input.extend([self.params[self.keys[i]][self._idx[i]][key] for key in self.binding[self.keys[i]]])
            self._next_idx()
            return input
        else: return None
        
    def _get_param_frame(self):
        param_list = []
        self._reset_config()
        param = self._next_param_list()
        while param != None:
            param_list.append(param)
            param = self._next_param_list()
        return pd.DataFrame(param_list, columns=self.param_keys)
        
    def get_csv(self, csv_url):
        self._get_param_frame().to_csv(csv_url, index=False)
        print('successfully generated grid search csv file\n')
    
    def _get_grid_search_param(keys, values): return dict(zip(keys, values))
    
    def _get_next_param(csv_url, param_list):
        df = shuffle(pd.read_csv(csv_url))
        if df.empty: return None
        param = grid_search_generator._get_grid_search_param(param_list, df.iloc[0].values)
        df.iloc[1 : ].to_csv(csv_url, index=False)
        return param
    
    def _recorder(values, record_csv, score_name_list, params, param_keys, label_list, confusion_matrices, initialize):
        head = list(param_keys)
        head.extend(score_name_list)
        if label_list != None: head.extend(label_list)
        
        content = []
        for x in param_keys: content.append(params[x])
        content.extend(values)
        if label_list != None:
            for x in label_list: content.append(confusion_matrices[x])
        if initialize:
            with open(record_csv, 'w') as f: pd.DataFrame([content], columns=head).to_csv(f, header=True, index=False)
        else:
            with open(record_csv, 'a') as f: pd.DataFrame([content], columns=head).to_csv(f, header=False, index=False)

        print_str = ''
        for i, j in enumerate(values): print_str += (score_name_list[i] + ':' + str(j) + ', ')
        print_str = print_str[ : -2] + '\n'
        print(print_str)
    
    def add_params(csv_url, param_dict, config_file_url='grid_search_config.json'):
        for key in param_dict.keys():
            grid_search = grid_search_generator(config_file_url=config_file_url)
            original = list(grid_search.params[key])
            param_values = [value for value in param_dict[key] if value not in original]
            grid_search.params[key] = param_values
            with open(csv_url, 'a') as f: grid_search._get_param_frame().to_csv(f, header=False, index=False)
            grid_search.params[key].extend(original)
            grid_search._save_config(config_file_url)
        
        print('successfully append the new parameters')
        
    def delete_params(csv_url, param_dict, config_file_url='grid_search_config.json'):
        csv = pd.read_csv(csv_url)
        grid_search = grid_search_generator(config_file_url)
        
        for key in param_dict.keys():
            for value in param_dict[key]:
                csv = csv[csv[key] != value]
        with open(csv_url, 'w') as f: csv.to_csv(f, index=False)
        
        for key in param_dict.keys():
            if key in grid_search.single_keys:
                grid_search.params[key] = [value for value in grid_search.params[key] if value not in param_dict[key]]
            else:
                for bkey in grid_search.binding_keys:
                    if key in grid_search.binding[bkey]:
                        grid_search.params[bkey] = [value for value in grid_search.params[bkey] if value[key] not in param_dict[key]]
                        break
        grid_search._save_config(config_file_url)
        print('successfully delete the parameters')
    
    def search(remain_csv, record_csv, model_run, other_model_dependency_dict, label_list=None,
               config_file_url='grid_search_config.json', X_y_split=True, val_size=0.1, shuffle=True):
        grid_search = grid_search_generator(config_file_url)
        param = grid_search_generator._get_next_param(remain_csv, grid_search.param_keys)
        header = True
        
        train_total = pd.read_csv(PATH+'cleaned_train.csv')
        test = pd.read_csv(PATH+'cleaned_test.csv')
        if X_y_split:
            train, val = train_test_split(train_total, test_size=val_size, shuffle=shuffle)
            X_train = train[param['train']]
            y_train = train[label_cols]
            X_val = val[param['train']]
            y_val = val[label_cols]

        while param != None:
            print(param)
            if X_y_split:
                scores, confusion_matrices = model_run(param=param, X=X_train, X_val=X_val, y=y_train, y_val=y_val,
                                                       other_model_dependency_dict=other_model_dependency_dict,
                                                       test=test[param['train']])
            else:
                scores, confusion_matrices = model_run(param=param, X=train_total[param['train']],
                                                       y=train_total[label_cols], test=test[param['train']],
                                                       other_model_dependency_dict=other_model_dependency_dict)
            grid_search_generator._recorder(scores, record_csv, grid_search.score_name_list, param,
                                            grid_search.param_keys, label_list, confusion_matrices, header)
            header = False
            param = grid_search_generator._get_next_param(remain_csv, grid_search.param_keys)
            
print('done')

done


In [3]:
def get_coefs(word,*arr): 
#     import pdb
#     pdb.set_trace()
    return word, np.asarray(arr[-300:], dtype='float32')

def glove_get_embedding_matrix(embedding_file, embed_size, max_features, tokenizer):
    embeddings_index = dict(get_coefs(*o.strip().split()) for o in open(embedding_file, encoding='utf8'))
    embeddings_index.pop('2000000', None)  # only needed when using lexvec
    word_index = tokenizer.word_index
    nb_words = min(max_features, len(word_index) + 1)
    embedding_matrix = np.zeros((nb_words, embed_size))
    for word, i in word_index.items():
        if i < max_features:
            embedding_vector = embeddings_index.get(word)
            if embedding_vector is not None: embedding_matrix[i] = embedding_vector
    return embedding_matrix, nb_words

def fasttext_get_embedding_matrix(embedding_file, embed_size, max_features, tokenizer):
    word_index = tokenizer.word_index
    ft_model = load_model(embedding_file)
    nb_words = min(max_features, len(word_index) + 1)
    embedding_matrix = np.zeros((nb_words, embed_size))
    for word, i in word_index.items():
        if i < max_features:
            embedding_vector = ft_model.get_word_vector(word).astype('float32')
            if embedding_vector is not None: embedding_matrix[i] = embedding_vector
    return embedding_matrix, nb_words

def get_embedding_matrix(embed_type, file, size, max_features, tokenizer):
    if embed_type == 'fasttext': return fasttext_get_embedding_matrix(file, size, max_features, tokenizer)
    else: return glove_get_embedding_matrix(file, size, max_features, tokenizer)

def get_rnn_model(params, tokenizer):
    embed_type = params['embed_type']
    embed_file = params['embed_file']
    embed_size = params['embed_size']
    embed_trainable = params['embed_trainable']
    recurrent_dropout = params['recurrent_dropout']
    max_len = params['max_len']
    cnn = params['cnn']
    dense = params['dense']
    
    loss = params['loss']
    label_len = params['label_len']
    max_features = params['max_features']
    gru_lstm = params['gru_lstm']
    lstm_units = int(params['lstm_units'])
    
    sequence_input = Input(shape=(max_len, ))
    embedding_matrix, inp_len = get_embedding_matrix(embed_type, embed_file, embed_size, max_features, tokenizer)
#     import pdb
#     pdb.set_trace()
    x = Embedding(inp_len, embed_size, weights=[embedding_matrix], trainable=embed_trainable)(sequence_input)
    x = SpatialDropout1D(0.2)(x)
    if gru_lstm == 'lstm':
        x = Bidirectional(LSTM(lstm_units, return_sequences=True, dropout=0.1, recurrent_dropout=0.1))(x)
    else:
        x = Bidirectional(GRU(lstm_units, return_sequences=True, dropout=0.1, recurrent_dropout=0.1))(x)
        #x = Bidirectional(GRU(lstm_units, return_sequences=True, dropout=0.1, recurrent_dropout=0.1))(x)
    if cnn:
        x = Conv1D(64, kernel_size=3, padding="valid", kernel_initializer="glorot_uniform")(x)
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    x = concatenate([avg_pool, max_pool]) 
    
    if dense:
        x = Dense(100, activation='relu')(x)
        x = Dropout(0.2)(x)
    
    preds = Dense(label_len, activation="sigmoid")(x)
    model = Model(sequence_input, preds)
    model.compile(loss=loss,optimizer=Adam(lr=1e-3),metrics=['accuracy'])
    
    return model

from keras.callbacks import Callback
class RocAucEvaluation(Callback):
    def __init__(self, validation_data=(), interval=1):
        super(Callback, self).__init__()

        self.interval = interval
        self.X_val, self.y_val = validation_data

    def on_epoch_end(self, epoch, logs={}):
        if epoch % self.interval == 0:
            y_pred = self.model.predict(self.X_val, verbose=0)
            score = roc_auc_score(self.y_val, y_pred)
            print("\n ROC-AUC - epoch: {:d} - score: {:.6f}".format(epoch+1, score))


def train_model(model, x_train, y_train, x_val, y_val, params, test):
    batch_size = params['batch_size']
    epochs = params['epochs']
    patience = params['patience']
    time_stamp = str(int(time()))
    model_file = '../../model/' + time_stamp + '.hdf5'
    pred_file = '../../model/' + time_stamp + '.csv'
    print(model_file)
    
    checkpoint = ModelCheckpoint(model_file, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
                                #(model_file, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
    earlystopping = EarlyStopping(monitor="val_loss", mode="min", patience=patience)
                                  #(monitor="val_acc", mode="max", patience=3)
    ra_val = RocAucEvaluation(validation_data=(x_val, y_val), interval = 1)
    callbacks_list = [checkpoint, earlystopping, ra_val]
    history = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs,\
                        validation_data=(x_val,y_val), callbacks=callbacks_list)
    
    # predict
    model.load_weights(model_file)
    y_pre = model.predict(x_val, verbose=1)
    pred = model.predict(test, verbose=1)
    
    submission = pd.read_csv(PATH + 'sample_submission.csv')
    submission[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]] = pred
    submission.to_csv(pred_file, index=False)
    
    # compute the scores
    best_epoch = history.history['val_loss'].index(min(history.history['val_loss']))
    val_loss = history.history['val_loss'][best_epoch]
    val_auc = roc_auc_score(y_val, y_pre)
    
    thres = 0.5
    def f(x):
        return (x > thres)*1
    
    M = {}
    y_pre = pd.DataFrame(y_pre, columns=label_cols)
    for i in label_cols:
        y_tmp = y_pre[i].apply(f)
        M[i] = confusion_matrix(y_val[i], y_tmp)
    
    return val_loss, best_epoch, val_auc, model_file, pred_file, M

def model_run(param, X, y, other_model_dependency_dict, test, X_val = None, y_val=None):
    
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1)
    
    tokenizer = text.Tokenizer(num_words=param['max_features'])
    tokenizer.fit_on_texts(X_train.values)
    tokenized_train = tokenizer.texts_to_sequences(X_train.values)
    tokenized_val = tokenizer.texts_to_sequences(X_val.values)
    x_train = sequence.pad_sequences(tokenized_train, maxlen=param['max_len'])
    x_val = sequence.pad_sequences(tokenized_val, maxlen=param['max_len'])
    tokenized_test = tokenizer.texts_to_sequences(test.values)
    test = sequence.pad_sequences(tokenized_test, maxlen=param['max_len'])
    
    model = get_rnn_model(param, tokenizer)
    val_loss, best_epoch, val_auc, model_file, pred_file, M = train_model(model, x_train, y_train,
                                                                          x_val, y_val, param, test)
    return [val_loss, best_epoch, val_auc, model_file, pred_file], M

print('done')

done


In [ ]:
# import numpy as np
# def get_coefs(word,*arr): 
#     try:
#         return word, np.asarray(arr, dtype='float32') 
#     except ValueError:
#         return 'nnnnnnnaaaaaaa@@!',np.zeros(300)
# file = '/home/kai/data/resources/FastText/wiki.en.vec'
# embeddings_index = dict(get_coefs(*o.strip().split()) for o in open(file, encoding='utf8')) 
 


In [ ]:
# params = {'max_features': [100000,150000], 
#           'epochs': [8],#,5,6], 
#           'batch_size': [128],64,32],
#           'max_len': [250,150],#int(np.round(mean_length + 2*std_length)), int(np.round(mean_length + 1*std_length))],
#           'dropout': [0.1],
#           'patience': [3],
#           'loss': ['binary_crossentropy'],
#           'label_len': [len(label_cols)],
#           'embed_trainable': [True, False],
#           'batch_normalization': [False],
#           'activation': ['relu']#, 'tanh', 'sigmoid'],
#           'lstm_activation': ['tanh'],
#           'lstm_units': [64, 128],#GRU, LSTM
#           'dense_units': [100],
#           'lstm_layer_size': [1],
#           'dense_layer_size': [1],
#           'embedding_param': [{'embed_file': '/home/kai/data/resources/glove/glove.840B.300d.txt', 'embed_size': 300, 'embed_type': 'glove'},
#                               {'embed_file': '/home/kai/data/resources/glove/glove.twitter.27B.200d.txt', 'embed_size': 200, 'embed_type': 'glove'},
#                               {'embed_file': '/home/kai/data/resources/FastText/wiki.en.bin', 'embed_size': 300, 'embed_type': 'fasttext'}]
#         }

params = {'train': ['comment_text', 'comment_text_cleaned'],
          'max_features': [100000],
          'epochs': [6],
          'batch_size': [128],
          'max_len': [150],
          'recurrent_dropout': [0.1],
          'patience': [2],
          'loss': ['binary_crossentropy'],
          'label_len': [len(label_cols)],
          'embed_trainable': [False],
          'cnn': [True, False],
          'lstm_units': [32, 128],
          'gru_lstm': ['gru', 'lstm'],
          'dense': [True, False],
#           'embedding_param': [{'embed_file': '../../data/glove.6B.50d.txt', 'embed_size': 50, 'embed_type': 'glove'}]
          'embedding_param': [{'embed_file': '/home/kai/data/resources/glove/glove.twitter.27B.200d.txt', 'embed_size': 200, 'embed_type': 'glove'},
                              {'embed_file': '/home/kai/data/resources/lexvec/lexvec.commoncrawl.300d.W.pos.vectors', 'embed_size': 300, 'embed_type': 'glove'},
                              {'embed_file': '/home/kai/data/resources/glove/glove.840B.300d.txt', 'embed_size': 300, 'embed_type': 'glove'},
                              {'embed_file': '/home/kai/data/resources/FastText/wiki.en.bin', 'embed_size': 300, 'embed_type': 'fasttext'}]
         }

bindings = {'embedding_param': ['embed_type', 'embed_file', 'embed_size']}

score_name_list = ['val_loss', 'best_epoch', 'val_auc', 'model_file', 'pred_file']

grid_search_csv_url = 'grid_search.csv'
grid_search_result_csv_url = 'grid_search_result.csv'
# param_class = grid_search_generator(params=params, bindings=bindings,
#                                     score_name_list=score_name_list, csv_url=grid_search_csv_url)

grid_search_generator.search(remain_csv=grid_search_csv_url, record_csv=grid_search_result_csv_url,
                             model_run=model_run, other_model_dependency_dict=None, X_y_split=True,
                             label_list=label_cols)

print('done')

{'max_features': 100000, 'embed_type': 'glove', 'gru_lstm': 'lstm', 'embed_size': 300, 'label_len': 6, 'lstm_units': 128, 'embed_file': '/home/kai/data/resources/glove/glove.840B.300d.txt', 'recurrent_dropout': 0.1, 'epochs': 8, 'embed_trainable': False, 'loss': 'binary_crossentropy', 'patience': 3, 'max_len': 150, 'train': 'comment_text', 'dense': True, 'batch_size': 128, 'cnn': True}
../../model/1520863673.hdf5
Train on 129251 samples, validate on 14362 samples
Epoch 1/8
129152/129251 [============================>.] - ETA: 0s - loss: 0.0617 - acc: 0.9788
Epoch 00001: val_loss improved from inf to 0.04649, saving model to ../../model/1520863673.hdf5

 ROC-AUC - epoch: 1 - score: 0.973952
129251/129251 [==============================] - 357s 3ms/step - loss: 0.0617 - acc: 0.9788 - val_loss: 0.0465 - val_acc: 0.9826
Epoch 2/8
129152/129251 [============================>.] - ETA: 0s - loss: 0.0455 - acc: 0.9828
Epoch 00002: val_loss improved from 0.04649 to 0.04346, saving model to ../.


 ROC-AUC - epoch: 3 - score: 0.984798
129251/129251 [==============================] - 361s 3ms/step - loss: 0.0468 - acc: 0.9822 - val_loss: 0.0442 - val_acc: 0.9828
Epoch 4/8
129152/129251 [============================>.] - ETA: 0s - loss: 0.0446 - acc: 0.9830
Epoch 00004: val_loss improved from 0.04422 to 0.04324, saving model to ../../model/1520869857.hdf5

 ROC-AUC - epoch: 4 - score: 0.987354
129251/129251 [==============================] - 360s 3ms/step - loss: 0.0446 - acc: 0.9830 - val_loss: 0.0432 - val_acc: 0.9831
Epoch 5/8
129152/129251 [============================>.] - ETA: 0s - loss: 0.0424 - acc: 0.9835
Epoch 00005: val_loss improved from 0.04324 to 0.04193, saving model to ../../model/1520869857.hdf5

 ROC-AUC - epoch: 5 - score: 0.988180
129251/129251 [==============================] - 357s 3ms/step - loss: 0.0425 - acc: 0.9835 - val_loss: 0.0419 - val_acc: 0.9835
Epoch 6/8
129152/129251 [============================>.] - ETA: 0s - loss: 0.0411 - acc: 0.9840
Epoch 00

129152/129251 [============================>.] - ETA: 0s - loss: 0.0370 - acc: 0.9856
Epoch 00008: val_loss did not improve

 ROC-AUC - epoch: 8 - score: 0.983408
153164/153164 [==============================] - 363s 2ms/step
val_loss:0.042568991778558746, best_epoch:6, val_auc:0.9833286617034419, model_file:../../model/1520875718.hdf5, pred_file:../../model/1520875718.csv

{'max_features': 100000, 'embed_type': 'glove', 'gru_lstm': 'gru', 'embed_size': 200, 'label_len': 6, 'lstm_units': 128, 'embed_file': '/home/kai/data/resources/glove/glove.twitter.27B.200d.txt', 'recurrent_dropout': 0.1, 'epochs': 8, 'embed_trainable': False, 'loss': 'binary_crossentropy', 'patience': 3, 'max_len': 150, 'train': 'comment_text_cleaned', 'dense': False, 'batch_size': 128, 'cnn': True}
../../model/1520879061.hdf5
Train on 129251 samples, validate on 14362 samples
Epoch 1/8
129152/129251 [============================>.] - ETA: 0s - loss: 0.0599 - acc: 0.9793
Epoch 00001: val_loss improved from inf to 0

Epoch 2/8
129152/129251 [============================>.] - ETA: 0s - loss: 0.0476 - acc: 0.9823
Epoch 00002: val_loss improved from 0.04845 to 0.04359, saving model to ../../model/1520885251.hdf5

 ROC-AUC - epoch: 2 - score: 0.982886
129251/129251 [==============================] - 367s 3ms/step - loss: 0.0476 - acc: 0.9823 - val_loss: 0.0436 - val_acc: 0.9830
Epoch 3/8
129152/129251 [============================>.] - ETA: 0s - loss: 0.0440 - acc: 0.9832
Epoch 00003: val_loss improved from 0.04359 to 0.04218, saving model to ../../model/1520885251.hdf5

 ROC-AUC - epoch: 3 - score: 0.984940
129251/129251 [==============================] - 364s 3ms/step - loss: 0.0440 - acc: 0.9832 - val_loss: 0.0422 - val_acc: 0.9835
Epoch 4/8
129152/129251 [============================>.] - ETA: 0s - loss: 0.0416 - acc: 0.9837
Epoch 00004: val_loss improved from 0.04218 to 0.04205, saving model to ../../model/1520885251.hdf5

 ROC-AUC - epoch: 4 - score: 0.986125
129251/129251 [======================